In [ ]:
from IPython.display import Image, HTML, display, IFrame

<center> 
<h1> CaNPAN Projects: Nova </h1>
<h2> Introduction </h2>
</center>

<center>
<h4>Nova explosions occur in close binary systems on white dwarfs accreting H-rich material from their main-sequence (MS), sub-giant (SG) or red-giant (RG) companions (a picture from slideplayer.com)</h4>
<img src="img/NovaBinarySystem.jpg", width=800>
</center>

<center>
<h4>HST images of Nova Cygni 1992 (V1974) taken 7 months apart in 1993-1994 (https://en.wikipedia.org/wiki/V1974_Cygni)</h4>
<img src="img/classicalnovae1.jpg", width=600>
</center>

<center>
<h4>In this project, we focus on H-burning nucleosynthesis in 1D hydrostatic nova models, rather than on nova photometry or spectroscopy at different wavelengths</h4>
</center>

The H-rich material accreted by the white dwarf is being compressed by the force of gravity and, as a result of this compression, it is getting heated up until H is ignited leading to a thermonuclear explosion. A large amount of heat released by this runaway H burning at the base of the accreted envelope triggers convection in the envelope and causes its expansion. It is interesting that a similar sequence of events, namely air intake, its compression, fuel
injection and combustion, finally followed by exaust of burning products, takes place in a four-stroke diesel engine (see the animation below). The similarity between the nova and diesel engine duty cycles is enhanced by the fact that a successfull nova explosion also needs an injection (mixing) of CO- or ONe-rich material from an underlying CO or ONe WD that serves as catalyst for H burning in the CNO cycle.

<center>
<h4>A mechanical analogue of novae is a four-stroke diesel engine</h4>     
<img src="img/diesel_v5.gif", width=600>
</center>

In [3]:
HTML('<iframe width="1000" height="700" src="https://www.youtube.com/embed/6J468JHtPuE" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

<center> 
<h4>
    The strength of a nova explosion (the maximum temperature of H burning and the envelope expansion speed) is amplified with
</h4>        
</center>

* an increase of the WD mass $M_\mathrm{WD}$ (a higher mass WD has a smaller radius, hence a deeper gravitational potential well, therefore the accreted material is stronger compressed); this also reduces the total accreted mass before the nova explosion

* a decrease of the WD central temperature $T_\mathrm{WD}$ (luminosity), which slows down the heating up of the compressed accreted material; this results in a higher accreted mass

* a decrease of the mass accretion rate $\frac{dM_\mathrm{acc}}{dt}$, which reduces the gravothermal energy release rate, thus also leading to a a higher accreted mass    